In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import tqdm as tqdm
from collections import defaultdict
import pickle as pickle

def get_tfidf(column):
	tfidf_vectorizer = TfidfVectorizer(stop_words ="english")
	tfidf_matrix = tfidf_vectorizer.fit_transform(column.values.astype(str))
	return tfidf_matrix

def find_similar(tfidf_matrix, index, top_n = 20):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)	

answers_file = "CS_answers.csv"
df = pd.read_csv(answers_file)
saved_column = df.Body #you can also use df['column_name']
saved_column.fillna(" ")
tfidf_answers = get_tfidf(saved_column)

answers_rec_dict = defaultdict(list)
for i in tqdm.tqdm(range(saved_column.shape[0])):
    for index, score in find_similar(tfidf_answers, i):
        if df['ParentId'][i] in answers_rec_dict:
            (answers_rec_dict[df['ParentId'][i]]).append((df['ParentId'][index],score))
        else:
            answers_rec_dict[df['ParentId'][i]] = [(df['ParentId'][index],score)]

save_obj(answers_rec_dict,"tf_idf_cs_answers_rec")

100%|██████████| 35056/35056 [50:08<00:00, 11.65it/s]


In [0]:

import pandas as pd
postlinksDF = pd.read_csv('postlinks.csv')

In [3]:
import tqdm 
#final list of dict to keep track of related post ids and answer ids for each question id
postLinks = []
#to keep track of questions that were already encountered
posts = [] 

for index, row in tqdm.tqdm(postlinksDF.iterrows()):
    temp = {'postID': None,'relatedID': [],'answerID': [] } 
    temp['postID'] = row['PostId']
    if(row['PostId'] in posts and len(posts)!=0):
        for i in range(0,len(posts)):
            if(row['PostId'] == postLinks[i]['postID']):
                postLinks[i]['relatedID'].append(row["RelatedPostId"])
                postLinks[i]['answerID'].append(row["Id"])
    else:  
        temp['relatedID'].append(row["RelatedPostId"])
        temp['answerID'].append(row["Id"])   
        posts.append(temp['postID'])
        postLinks.append(temp)

9855it [02:14, 73.14it/s]


In [0]:
import pickle as pickle

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

tf_idf_cs_ques_rec = load_obj("tf_idf_cs_ques_rec")
tf_idf_answers_rec = load_obj("tf_idf_cs_answers_rec")

In [0]:
def evaluate(recommended):
    score  = 0
    for i in range(0,len(postLinks)):
        present = 0 
        top = 0
        count = len(postLinks[i]['relatedID'])
        if postLinks[i]['postID'] in recommended.keys():
            idx = 0
            for rec in recommended[postLinks[i]['postID']]:
                idx+=1
                if rec[0] in postLinks[i]['relatedID']:
                    present+=1
                    if (idx >= int(len(postLinks[i]['relatedID'])/2)):
                        top+=1
            if(present):
                score+= (0.3* (present/count)+ 0.7*(top/present))
    return (score/len(postLinks))

In [11]:
evaluate(tf_idf_answers_rec)*100

18.681259372435896